## Задание 08
В задании не будет подробных инструкций. Ваша задача - построить классификатор с как можно лучшим качеством (AUC-ROC). 

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

#from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

Будем работать с данными по авиарейсам в США. Задача  -  предсказать задержку вылета более 15 минут (задача бинарной классификации).

Признаки:

* Month, DayofMonth, DayOfWeek, месяц, день месяца и день недели
* DepTime, время отправления
* UniqueCarrier, код перевозчика
* Origin, место вылета
* Dest, место назначения
* Distance, расстояние между аэропортами вылета и прилета
* dep_delayed_15min, просрочка вылета на 15 и более минут (целевой признак)


In [47]:
df = pd.read_csv('flight_delays.csv')

In [48]:
df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`:

In [49]:
X, y = df[['Distance', 'DepTime']].values, df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_train1, X_test1, y_train1, y_test1 = train_test_split(X,y,test_size=0.3,random_state=17)

In [50]:
lr = LogisticRegression( solver='lbfgs',random_state=17)

lr.fit(X_train, y_train)
y_pred = lr.predict_proba(X_test)[:, 1]

print('AUC-ROC:',roc_auc_score(y_test, y_pred))

lr2 = LinearRegression()
lr2.fit(X_train, y_train )

AUC-ROC: 0.5816110169880349


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

####  Критерии оценивания:

#### 1. Предобработка данных  - 3 балла 

Предобработка включает one-hot кодирование категориальных признаков, заполнение пропусков (если есть),  генерацию новых признаков (например, вместо Origin и Dest можно ввести признак маршрут: Origin-Dest)

In [51]:
#генерируем признак Origin-Dest
df['Origin-Dest'] = df['Origin']+ ' '+df['Dest']
df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,Origin-Dest
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N,ATL DFW
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N,PIT MCO
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N,RDU CLE
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N,DEN MEM
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y,MDW OMA


In [52]:
#преобразуем категориальные признаки
le= LabelEncoder()
df['Month'] = le.fit_transform(df['Month'])
df['DayofMonth'] = le.fit_transform(df['DayofMonth'])
df['DayOfWeek'] = le.fit_transform(df['DayOfWeek'])
df['Origin-Dest'] = le.fit_transform(df['Origin-Dest'])
df['UniqueCarrier'] = le.fit_transform(df['UniqueCarrier'])
#удаляем ненужные признаки
df = df.drop('Origin', axis=1)
df = df.drop('Dest', axis=1)
df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Distance,dep_delayed_15min,Origin-Dest
0,10,13,6,1934,0,732,N,152
1,6,12,2,1548,18,834,N,3527
2,11,11,4,1422,20,416,N,3619
3,2,17,5,1015,15,872,N,1181
4,1,28,5,1828,19,423,Y,2681


In [53]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('dep_delayed_15min',axis=1), 
                                                    df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values,
                                                    stratify=df['dep_delayed_15min'], random_state=20)
dtrain=xgb.DMatrix(X_train, y_train)
dtest =xgb.DMatrix(X_test, y_test)

In [54]:
params = { 'objective':'binary:logistic', 'max_depth': 5, 'silent': 1, 'eta': 1} 
num_rounds = 1000
xgb_model = xgb.train(params, dtrain, num_rounds)
preds_prob = xgb_model.predict(dtest)
predicted_labels = preds_prob > 0.5 
print('AUC-ROC:', roc_auc_score(y_test, predicted_labels))

AUC-ROC: 0.6003612013917409


#### 2. Настройка параметров обучения - (4 балла) 
Настройка включает определение гиперпараметров бустинга (предполагается, что вы будете использовать XGBoost) на кросс-валидации: глубина деревьев, темп обучения, число деревьев и т.д. 

#### 3. Использование стекинга, отбора признаков и другое - (3 балла)

Например, можно реализовать простейшую схему стекинга - блендинг:
* обучите логистическую регрессию и градиентный бустинг;
* постройте линейную смесь ответов логистической регрессии и градиентного бустинга вида 
$$p=w_1∗p_{lr}+(1−w_1)∗p_{xgb},$$ 
где  $p_{lr}$  – предсказанные логистической регрессией вероятности класса 1,  $p_{xgb}$ – бустинга. 

In [55]:
xgb_model = xgb.XGBClassifier(objective='binary:logistic')

xgb_grid = GridSearchCV(xgb_model, {'max_depth': [2,3,5,7],'eta': [0.2,0.5,1]}, verbose=1) 

xgb_grid.fit(X_train, y_train) 
print(xgb_grid.best_score_) 
print(xgb_grid.best_params_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  1.5min finished


0.8193466666666667
{'eta': 0.2, 'max_depth': 5}


In [56]:
predicted_labels = xgb_grid.predict(X_test)
print('AUC-ROC:', roc_auc_score(y_test, predicted_labels))

AUC-ROC: 0.5434673380471565


In [65]:
#Задание 3
new_df = pd.DataFrame()
new_df['xgb'] = xgb_grid.predict_proba(X_train)[:, 1]
new_df['lr'] = lr.predict_proba(X_train1)[:, 1]

ValueError: X has 2 features per sample; expecting 7

In [62]:
Lr = LogisticRegression()
Lr.fit(new_df, y_train)
new_test_df = pd.DataFrame()
new_test_df['xgb'] = xgb_grid.predict_proba(X_test)[:, 1]
new_test_df['lr'] = lr.predict_proba(X_test1)[:, 1

SyntaxError: unexpected EOF while parsing (<ipython-input-62-32e7e8b839da>, line 5)

In [66]:
pred_ = Lr.predict_proba(new_test_df)[:, 1]
print('AUC-ROC:',roc_auc_score(y_test, pred_))

NameError: name 'Lr' is not defined